In [1]:
%load_ext autoreload
%autoreload 2

from pathlib import Path
import os

import numpy as np
from dotenv import find_dotenv, load_dotenv
from sentence_transformers import SentenceTransformer

from podology.data.Episode import Episode
from podology.data.EpisodeStore import EpisodeStore
from podology.data.Transcript import Transcript
from podology.search.utils import make_index_name
import config

# set cwd one level up:
load_dotenv(find_dotenv())
os.chdir(Path.cwd().parent)

In [2]:
episode_store = EpisodeStore()

In [3]:
ep = episode_store["Ug6QC"]
ep

Episode (id 'Ug6QC')
  Title: TEST1
  Audio: ✅️
  TrScr: ✅️
  URL:   https://traffic.libsyn.com/secure/knowledgefight/test1.mp3?dest-id=461805
  Publ:  2025-09-07

In [27]:
t =Transcript(ep)

In [26]:
t._chunked(min_words=20, max_words=50, overlap=.9)

[{'text': "Maybe, you know what, maybe Alex is too. Maybe this is an unexplored thing that we haven't talked about. He's definitely not. He hates songs. No, definitely not. Yeah. I think it's fine if you don't know all the words. Maybe Alex's brand does not allow it.",
  'start': 0.031,
  'end': 13.73},
 {'text': "Maybe this is an unexplored thing that we haven't talked about. He's definitely not. He hates songs. No, definitely not. Yeah. I think it's fine if you don't know all the words. Maybe Alex's brand does not allow it. He's too busy talking over the song.",
  'start': 2.514,
  'end': 16.834},
 {'text': "He's definitely not. He hates songs. No, definitely not. Yeah. I think it's fine if you don't know all the words. Maybe Alex's brand does not allow it. He's too busy talking over the song. So if you're talking over the song, you just think every word is hood.",
  'start': 4.918,
  'end': 20.44},
 {'text': "Maybe Alex's brand does not allow it. He's too busy talking over the song.

In [34]:
t.chunks(chunk_attrs=["start", "end"], overlap=.2, min_words=20, max_words=50, episode_attrs=["eid"])

[{'eid': 'Ug6QC',
  'start': 0.031,
  'end': 13.73,
  'text': "Maybe, you know what, maybe Alex is too. Maybe this is an unexplored thing that we haven't talked about. He's definitely not. He hates songs. No, definitely not. Yeah. I think it's fine if you don't know all the words. Maybe Alex's brand does not allow it."},
 {'eid': 'Ug6QC',
  'start': 9.004,
  'end': 20.44,
  'text': "I think it's fine if you don't know all the words. Maybe Alex's brand does not allow it. He's too busy talking over the song. So if you're talking over the song, you just think every word is hood."},
 {'eid': 'Ug6QC',
  'start': 16.855,
  'end': 28.671,
  'text': "So if you're talking over the song, you just think every word is hood. I get it if you're like, you know, some other song, like a rock song. Maybe if you have the wrong word, that's happened to be a thousand times. Tons of times. Yeah."},
 {'eid': 'Ug6QC',
  'start': 24.826,
  'end': 34.493,
  'text': "Maybe if you have the wrong word, that's happ